# Position Decomposition

Decomposing a wavefront into Gausslets turns out to be a somewhat involved procedure. 
There are a number of rules to follow:

* The new Gausslets must have direction orthogonal to the wavefront surface (i.e. their direction is the gradient of the phase)
* The Gausslets must have curvature that matches the input wavefront
* The sum of the E-field contributions of the Gausslets should equal the input E-field (ideally, everywhere)

The last of these is a bit of a pain. The Gausslets we create are going to overlap, in order to give a nice
smooth E-field. However, this means they are not independent. The E-field at the origin of one Gausslet 
also include the contribution from the surrounding ones. For a hexagonal-type arrangement, the central mode contributes something like 65% of the amplitude. The immediate neighbours add up to about 30% and the next 
12 further out contribute something like 3% and so on (these are guestimate numbers; hopefully we can 
obtain the concrete values later on in this document). I am expecting that for most applications, the nearest two rings will give sufficient accuracy. 

In [8]:
import sys
sys.path.append("..")
import numpy
from matplotlib import pyplot as pp
from raypier.core.gausslets import calc_mode_curvature, eval_Efield_from_gausslets, \
        build_interaction_matrix
from raypier.core.ctracer import ray_dtype, GaussletCollection, Gausslet, Ray

We being by constructing an input mode, being a single Gaussian beam directed along the z-axis and
converging to a beam-waist at (0,0,50).

Let the beam-radius be 10 wavelengths and the wavelength be 1um.

In [12]:
wavelength = 1.0

wavelengths = numpy.array([wavelength], 'd')

ray = Ray(origin = (0,0,0),
          direction = (0,0,1),
          E_vector = (1,0,0),
          E1_amp = 1.0,
          E2_amp = 0.0)

gausslet = Gausslet(base_ray=ray)
gc = GaussletCollection(1)
gc.add_gausslet(gausslet)

radius_mm = wavelength * 10 / 1000.0
working_dist = 50.0
gc.config_parabasal_rays(wavelengths, radius_mm, working_dist)


In [14]:
print(gc.para_origin)

[[[ 1.59154943  0.01        0.        ]
  [-1.59154943  0.01        0.        ]
  [-0.80443497  1.37332224  0.        ]
  [ 0.78711446 -1.38332224  0.        ]
  [-0.78711446 -1.38332224  0.        ]
  [ 0.80443497  1.37332224  0.        ]]]


We can see that our Gaussian beam is roughly 1.6mm radius at the origin.

We are going to decomposte the wavefront at the origin into modes with radius 50um, with radius up to
2mm. This means a resolution of 20.

In [ ]:
input_rays = gc
radius = 2.0
resolution = 20.0

In [ ]:
spacing = radius / resolution
wavelengths = input_rays.wavelengths

if len(numpy.unique(wavelengths)) > 1:
    raise ValueError("Can't decompose wavefront with more than one wavelength present.")

wavelength = wavelengths[0]

origin = numpy.asarray(origin)
direction = normaliseVector(numpy.asarray(direction))
axis1 = normaliseVector(numpy.asarray(axis1))
axis2 = numpy.cross(axis1, direction)
axis1 = numpy.cross(axis2, direction)

###In this version, we will evaluate the field on a cartesian grid
_radius = radius * (1 + 1./resolution)
x_ = y_ = numpy.linspace(-_radius,_radius, resolution*2)

x,y = numpy.meshgrid(x_, y_)

origins = origin[None,:] + x.reshape(-1,)[:,None]*axis1[None,:] + y.reshape(-1)[:,None]*axis2[None,:]

E_field = eval_Efield_from_gausslets(input_rays, origins, wavelengths)

### Project onto local axes to get orthogonal polarisations and choose the one with the most power
E1_amp = (E_field*axis1[None,:]).sum(axis=1)
E2_amp = (E_field*axis2[None,:]).sum(axis=1)

P1 = (E1_amp.real**2 + E1_amp.imag**2).sum()
P2 = (E2_amp.real**2 + E2_amp.imag**2).sum()

if P1 > P2:
    ### Always in the range -pi to +pi
    phase = numpy.arctan2(E1_amp.imag, E1_amp.real)
else:
    phase = numpy.arctan2(E2_amp.imag, E2_amp.real)

nx = len(x_)
ny = len(y_)
phase.shape = (nx, ny)

if curvature is not None:
    R = 1000*curvature/wavelength
    sph = R - numpy.sqrt(R*R - x*x - y*y) - numpy.pi
    phase = phase - sph
    phase = phase%(2*numpy.pi)
    phase = phase - numpy.pi
else:
    sph = 0

uphase = unwrap2d(phase, anchor=(nx//2, ny//2)) + sph

wavefront = RectBivariateSpline(x_, y_, uphase*wavelength/(2000*numpy.pi))

### Now make a hex-grid of new ray start-points
rx,ry, nb = make_hexagonal_grid(radius, spacing=spacing, connectivity=True)

N = len(rx)

origins = origin[None,:] + rx[:,None]*axis1[None,:] + ry[:,None]*axis2[None,:]

### First derivatives give us the ray directions
dx = wavefront(rx,ry,dx=1, grid=False)
dy = wavefront(rx,ry,dy=1, grid=False)

### Get second derivatives, to get wavefront curvature
dx2 = wavefront(rx,ry,dx=2, grid=False)
dy2 = wavefront(rx,ry,dy=2, grid=False)
dxdy = wavefront(rx,ry,dx=1,dy=1, grid=False)

### for direction, d:
### d^z gives a unit vector in the xy plane, called D
### D^d gives an orthogonal vector, called G
### A point r on surface is represented as (r.D, r.G)
### x' = (r-o).G, y' = (r-o).D, z' = (r-o).d
### dz'/dx' = 0 by definition

###Convert to A,B,C coefficients
A,B,C,x,y,z = calc_mode_curvature(rx, ry, dx, dy, dx2, dy2, dxdy)

max_spacing = spacing *2
data, (xi, xj) = build_interaction_matrix(rx, ry, A, B,  C, 
                                          x, y, z,
                                          wavelength, spacing, 
                                          max_spacing, blending)

M = coo_matrix( (data,(xi,xj)), shape=(N,N)) #I think M should be Hermitian

E_in = eval_Efield_from_gausslets(input_rays, origins, wavelengths) #should be a (N,3) array of complex values

E_out_x = lsqr(M, E_in[:,0])
E_out_y = lsqr(M, E_in[:,1])
E_out_z = lsqr(M, E_in[:,2])

ray_data = numpy.zeros(N, dtype=ray_dtype)

### Now need to construct the GaussletCollection.